[View in Colaboratory](https://colab.research.google.com/github/Yashwardhankaul/move37/blob/master/policy_and_value_iteration.ipynb)

Installing OpenAI gym

In [26]:
!pip install gym

Importing Dependencies

In [0]:
from time import time
import numpy as np
import gym

Function for running the policy and calculating the reward

In [0]:
def runPolicy(env,policy):
  state = env.reset()
  done = False
  
  totalReward = 0
  while not done:
    state, reward, done, _= env.step(policy[state])
    totalReward += reward
  return totalReward

Function for evaluating the policy

In [0]:
def evaluatePolicy(env, policy, iterations):
  totalRewards = 0
  for i in range(iterations):
    totalRewards += runPolicy(env, policy)
  return totalRewards/iterations

Functions common for both value iteration and policy iteration

In [0]:
eps = 1e-10

def constructGreedyPolicy(env, values, gamma):
  policy = np.zeros(env.env.nS)
  for s in range(env.env.nS):
    returns = [
        sum(p * (r + gamma * values[ns])
            for p, ns, r, _ in env.env.P[s][a])
        for a in range(env.env.nA)
    ]
    policy[s] = np.argmax(returns)
  
  return policy

def computeStateValues(env, gamma, policy = None, selectBest = False):
  if policy is None and not selectBest:
    raise 'When using computeStateValues either specify policy or pass selectBest=True!'
  if policy is not None and selectBest:
    raise 'You cannot use policy and selectBest at the same time'
  
  values = np.zeros(env.env.nS)
  while True:
    nextValues = values.copy()
    for s in range(env.env.nS):
      if policy is not None:
        action = policy[s]
        nextValues[s] = sum(p * (r + gamma * values[ns]) for p, ns, r, _ in env.env.P[s][action])
      else:
        nextValues[s] = max(
            sum(p * (r + gamma * values[ns])
                for p, ns, r, _ in env.env.P[s][a])
            for a in range(env.env.nA)
        )
      
    diff = np.fabs(nextValues - values).sum()
    values = nextValues
    if diff <= eps:
      break
  return values

Value Iteration

In [0]:
def valueIteration(env, gamma):
  stateValues = computeStateValues(env, gamma, selectBest = True)
  policy = constructGreedyPolicy(env, stateValues, gamma)
  return policy

Policy Iteration

In [0]:
def randomPolicy(env):
  return np.random.choice(env.env.nA, size=(env.env.nS))

def policyIteration(env,gamma):
  policy = randomPolicy(env)
  while True:
    stateValues = computeStateValues(env,gamma,policy)
    nextPolicy = constructGreedyPolicy(env,stateValues,gamma)
    if np.all(policy == nextPolicy):
      break
    policy = nextPolicy
    
  return policy

Testing the Methods

In [50]:
evaluateIterations = 1000

def solveEnv(env, methods, envName):
  print(f'Solving environment {envName}')
  for method in methods:
    name, f, gamma = method
    tstart = time()
    policy = f(env, gamma)
    tend = time()
    print(f'It took {tend - tstart} seconds to compute a policy using "{name}" with gamma={gamma}')
    
    score = evaluatePolicy(env, policy, evaluateIterations)
    print(f'Policy average reward is {score}')
  
  
methods = [
  ('Value Iteration', valueIteration, 0.9),
  ('Policy Iteration', policyIteration, 0.9),
  ('Value Iteration', valueIteration, 0.98),
  ('Policy Iteration', policyIteration, 0.98),
  ('Value Iteration', valueIteration, 1),
  ('Policy Iteration', policyIteration, 1),
]

frozenLake4 = gym.make('FrozenLake-v0')
solveEnv(frozenLake4, methods, 'Frozen Lake 4x4')

Solving environment Frozen Lake 4x4
It took 0.03473520278930664 seconds to compute a policy using "Value Iteration" with gamma=0.9
Policy average reward is 0.734
It took 0.05277681350708008 seconds to compute a policy using "Policy Iteration" with gamma=0.9
Policy average reward is 0.75
It took 0.09992027282714844 seconds to compute a policy using "Value Iteration" with gamma=0.98
Policy average reward is 0.724
It took 0.12077093124389648 seconds to compute a policy using "Policy Iteration" with gamma=0.98
Policy average reward is 0.715
It took 0.1971590518951416 seconds to compute a policy using "Value Iteration" with gamma=1
Policy average reward is 0.715
It took 0.09698224067687988 seconds to compute a policy using "Policy Iteration" with gamma=1
Policy average reward is 0.751


In [51]:
frozenLake8 = gym.make('FrozenLake8x8-v0')
solveEnv(frozenLake8, methods,'Frozen Lake 8x8')

Solving environment Frozen Lake 8x8
It took 0.12558817863464355 seconds to compute a policy using "Value Iteration" with gamma=0.9
Policy average reward is 0.739
It took 0.1747450828552246 seconds to compute a policy using "Policy Iteration" with gamma=0.9
Policy average reward is 0.75
It took 0.3746812343597412 seconds to compute a policy using "Value Iteration" with gamma=0.98
Policy average reward is 0.86
It took 0.5252554416656494 seconds to compute a policy using "Policy Iteration" with gamma=0.98
Policy average reward is 0.836
It took 1.178382396697998 seconds to compute a policy using "Value Iteration" with gamma=1
Policy average reward is 0.9
It took 3.532578706741333 seconds to compute a policy using "Policy Iteration" with gamma=1
Policy average reward is 0.88
